In [1]:
import pandas as pd

In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
study_group = [2,3]
# Assuming your DataFrame is named `dataframe_with_glucose_values_diabetes`
df =pd.read_csv("dataset_progression_analysis_"+str(study_group[0])+"_"+str(study_group[1])+".csv",index_col=0)
df.drop(columns=['participant_id',
       'study_group_id'], inplace=True)
print(df.columns)
df_train = df[df['recommended_split'] == 'train']
df_val   = df[df['recommended_split'] == 'val']
df_test  = df[df['recommended_split'] == 'test']
#df_test  = df[df['recommended_split'].isin(['val', 'test'])]
train_df = df_train.reset_index(drop=True)
val_df   = df_val.reset_index(drop=True)
test_df  = df_test.reset_index(drop=True)
train_df.drop(columns=['recommended_split'], inplace=True)
val_df.drop(columns=['recommended_split'], inplace=True)
test_df.drop(columns=['recommended_split'], inplace=True)
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

Index(['Creatinine (mg/dL)', 'mean_glucose', 'Troponin-T (ng/L)',
       'hyper_time_pcts', 'nocturnal_hypoglycemias', 'BUN (mg/dL)',
       'mlcsodlog, OD: Log Contrast Sensitivity',
       'mhoccur_rnl, Kidney problems',
       'mlcsodfcl, OD: Value of final correct letter',
       'via3, Because of your eyesight, how much difficul', 'CRP - HS (mg/L)',
       'plcsoslog, OS: Log Contrast Sensitivity', 'Albumin (g/dL)',
       'via1, How much difficulty, if any, do you have in',
       'plcsosfcl, OS: Value of final correct letter', 'HbA1c (%)',
       'recommended_split', 'healthy'],
      dtype='object')
(276, 17)
(72, 17)
(79, 17)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.metrics import roc_auc_score
import ast
col = "healthy"
feats_to_keep = [0] * 7
result_acc= [0] * 7
result_ba = [0] * 7
result_auc= [0] * 7


if study_group == [2,3]:
    # Blood test only features
    feats_to_keep[0] = ['HbA1c (%)']
    feats_to_keep[1] = ['HbA1c (%)']   
    feats_to_keep[2] = ['HbA1c (%)',  'Creatinine (mg/dL)', 'Troponin-T (ng/L)', 'BUN (mg/dL)','Albumin (g/dL)',
           'CRP - HS (mg/L)'] 
    # CGM 
    feats_to_keep[3] = [ 
             'hyper_time_pcts',
           'mean_glucose', 'nocturnal_hypoglycemias']
    
    # CGM + non blood test features
    feats_to_keep[4] = [ 
               'hyper_time_pcts',
           'mean_glucose', 'nocturnal_hypoglycemias',
            'mhoccur_rnl, Kidney problems',
           'via3, Because of your eyesight, how much difficul',
           'mlcsodlog, OD: Log Contrast Sensitivity',
           'mlcsodfcl, OD: Value of final correct letter',
           'plcsosfcl, OS: Value of final correct letter',
           'via1, How much difficulty, if any, do you have in', 
           'plcsoslog, OS: Log Contrast Sensitivity', ]
    
    feats_to_keep[5] = feats_to_keep[4]

if study_group ==[0,1]:
    feats_to_keep[0] = ['HbA1c (%)']
    feats_to_keep[1] = ['HbA1c (%)','Glucose (mg/dL)','INSULIN (ng/mL)']
    feats_to_keep[2] = ['HbA1c (%)','Glucose (mg/dL)','INSULIN (ng/mL)']
    feats_to_keep[3] = [ 
             'hyper_time_pcts', 'spike_resolutions',
           'mean_glucose', 'nocturnal_hypoglycemias']
    feats_to_keep[4] = [ 
             'hyper_time_pcts', 'spike_resolutions',
            'mean_glucose', 'nocturnal_hypoglycemias',
            'mhoccur_obs, Obesity', 
            'bmi_vsorres, BMI', 
            'waist_vsorres, Waist Circumference (cm)']
    feats_to_keep[5] = [ 
             'hyper_time_pcts', 'spike_resolutions',
             'mean_glucose', 'nocturnal_hypoglycemias',
             'mhoccur_rnl, Kidney problems',
             'mhoccur_hbp, High blood pressure', 'mhoccur_obs, Obesity', 'dmlsugar, When reflecting on your typical eating ',
             'bmi_vsorres, BMI', 'dmlact, What would you consider your typical acti', 
             'waist_vsorres, Waist Circumference (cm)', 'pxne7, The crime rate in my neighborhood makes it',
             'fh_dm2pt, Were either of your parents diagnosed w']
if study_group ==[1,2]:
    feats_to_keep[0] = ['HbA1c (%)']
    feats_to_keep[1] = ['HbA1c (%)','Glucose (mg/dL)']
    # Blood test only features
    feats_to_keep[2] = ['HbA1c (%)','Glucose (mg/dL)', 
     'LDL Cholesterol Calculation (mg/dL)',
     'HDL Cholesterol (mg/dL)',
     'Total Cholesterol (mg/dL)']
    # CGM 
    feats_to_keep[3] = [ 
             'hyper_time_pcts', 'spike_resolutions',
           'mean_glucose',  'expected_daily_spikes', 'nocturnal_hypoglycemias']
    # CGM + basic biometric features
    feats_to_keep[4] = [ 
             'hyper_time_pcts', 'spike_resolutions',
           'mean_glucose',  'expected_daily_spikes',
           'nocturnal_hypoglycemias',
           'mhoccur_hbp, High blood pressure',
           'whr_vsorres, Waist to Hip Ratio (WHR)',
           'waist_vsorres, Waist Circumference (cm)', 
           'pulse_vsorres, Heart Rate (bpm)']
    # CGM + non blood test features
    feats_to_keep[5] = [ 
           'hyper_time_pcts', 'spike_resolutions',
           'mean_glucose',  'expected_daily_spikes',
           'nocturnal_hypoglycemias',
           'mhoccur_hbp, High blood pressure',
           'whr_vsorres, Waist to Hip Ratio (WHR)',
           'waist_vsorres, Waist Circumference (cm)', 
           'pulse_vsorres, Heart Rate (bpm)',
           'fh_dm2pt, Were either of your parents diagnosed w',
           'via2, How much difficulty, if any, do you have re',
     ]

feats_to_keep[6] = train_df.columns[:-1]

for idx in range(len(feats_to_keep)):
    #print(feat_to_keep_df['feats'][idx])
    #"mhterm_dm2, Type II Diabetes"
    X_train =  train_df[feats_to_keep[idx]]
    y_train = train_df[col]
    X_val =  val_df[feats_to_keep[idx]]
    y_val = val_df[col]
    X_test =  test_df[feats_to_keep[idx]]
    y_test = test_df[col] 
    print(X_test.columns)
    # Train Random Forest
    rf = RandomForestClassifier(n_estimators=256)
    rf.fit(X_train, y_train)
    
    # Predict
    y_pred = rf.predict(X_test)
    y_prob = rf.predict_proba(X_test)[:, 1]  # Probability for class 1
    
    # Evaluate
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    # ROC AUC
    auc = roc_auc_score(y_test, y_prob)
    print(f"ROC AUC Score: {auc:.4f}")
    result_auc[idx] = auc
    result_acc[idx] = accuracy_score(y_test, y_pred)
    result_ba[idx] = balanced_accuracy_score(y_test, y_pred)

Index(['HbA1c (%)'], dtype='object')
Accuracy: 0.5063291139240507
Confusion Matrix:
 [[39  2]
 [37  1]]
Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.95      0.67        41
           1       0.33      0.03      0.05        38

    accuracy                           0.51        79
   macro avg       0.42      0.49      0.36        79
weighted avg       0.43      0.51      0.37        79

ROC AUC Score: 0.5276
Index(['HbA1c (%)'], dtype='object')
Accuracy: 0.5063291139240507
Confusion Matrix:
 [[39  2]
 [37  1]]
Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.95      0.67        41
           1       0.33      0.03      0.05        38

    accuracy                           0.51        79
   macro avg       0.42      0.49      0.36        79
weighted avg       0.43      0.51      0.37        79

ROC AUC Score: 0.5507
Index(['HbA1c (%)', 'Creatinine (mg/dL)', 'Tro

In [4]:
# Create dataframe
import os
os.makedirs("results", exist_ok=True)
result_df = pd.DataFrame({'feats':feats_to_keep,'ba':result_ba, 'acc':result_acc, 'auc':result_auc})
result_df.to_csv('results\\random_forest_'+str(study_group[0])+"_"+str(study_group[1])+'_5.csv')
result_df.head(7)

,feats,ba,acc,auc
0,[HbA1c (%)],0.488768,0.506329,0.527599
1,[HbA1c (%)],0.488768,0.506329,0.550706
2,"[HbA1c (%), Creatinine (mg/dL), Troponin-T (ng...",0.632542,0.645570,0.697047
3,"[hyper_time_pcts, mean_glucose, nocturnal_hypo...",0.517009,0.531646,0.591784
4,"[hyper_time_pcts, mean_glucose, nocturnal_hypo...",0.514121,0.531646,0.736200
5,"[hyper_time_pcts, mean_glucose, nocturnal_hypo...",0.540436,0.556962,0.740372
6,"Index(['Creatinine (mg/dL)', 'mean_glucose', '...",0.606226,0.620253,0.787548
